<div style="background-color:#008000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>GOOD NEWS:</strong> If we change the number of channels from 166x166x30 to 1 it runs (yep, I know it looks like a big diff but I think it should be 1 (not too sure)) 
</div>


<div style="background-color:#008000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>THEREFORE IGNORE ALL THE WARNINGS BELOW:</strong>
</div>


<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> Do not run in vs code! My computer just rebooted 
</div>


<div style="background-color:#FFA500; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>However:</strong> You can run everything until model = RCNN() in 2. Define the Neural Network 
</div>


<div style="background-color:#FFA500; color:#ffffff; padding:10px; border-radius:5px;">
    Also, the second time I ran it, my laptop just crashed for a few minutes, but didn't reboot, so maybe if you have a better laptop it works in yours 
</div>


## Some notes of things to add:

- We are minimizing loss, however we want to maximize F1 score. Do that in the cross validation. How? No f clue

# 0. Load Modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms, datasets
import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import os
import sys

In [2]:
# For Google Colab, mount Google Drive, for local environments, get local path (github)

# Change with the appropriate path. Log in into Drive and create the folders with the data

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    folder0_path = '/content/drive/My Drive/solar_jets/data0'
    folder1_path = '/content/drive/My Drive/solar_jets/data1'
else:
    # For local environments like VS Code
    folder0_path = './data0'
    folder1_path = './data1'

# 1. Very naive Model

## 1.1. Prepare the dataset

#### Create the class

In [3]:
class NPZDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir

        # in self.files, only add f if it has 30 samples
        self.files = [f for f in os.listdir(data_dir) if self._check_file_shape(f)]

    # Check if the file has 30 samples, if not, don't include it in self.files (see above)
    def _check_file_shape(self, file):
        file_path = os.path.join(self.data_dir, file)
        data = np.load(file_path)['arr_0']
        return data.shape == (166, 166, 30)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)['arr_0']

        # Rearrange dimensions to (30, 166, 166) for PyTorch (insted of (166, 166, 30))
        data = np.moveaxis(data, -1, 0)
        
        # assign label 1 if data is from data1, 0 if from data0
        label = 1.0 if 'data1' in self.data_dir else 0.0
        return torch.from_numpy(data).float(), label

#### Run to get the Data

In [4]:
train_data1 = NPZDataset(folder1_path)
train_data0 = NPZDataset(folder0_path)
train_data = torch.utils.data.ConcatDataset([train_data1, train_data0])

Below we'll see how train_data can be treated just like lists

- One element in the list for each sequence of images
- Each element of the list has a tuple of two elements, the first the array (166,30,30) and the second the label

In [5]:
print("samples y=1: ",len(train_data1))
print("samples y=0: ",len(train_data0))
print("samples total ",len(train_data))

samples y=1:  117
samples y=0:  10
samples total  127


In [6]:
print(f"Sample {0}: Shape {train_data1[0][0].shape}, Label: {train_data1[0][1]}")
print(f"Sample {0}: Shape {train_data0[0][0].shape}, Label: {train_data0[0][1]}")

Sample 0: Shape torch.Size([30, 166, 166]), Label: 1.0
Sample 0: Shape torch.Size([30, 166, 166]), Label: 0.0


#### Get the Minibatch

In [7]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

See that we have 4 different minibatches, each with 32 samples

In [8]:
print("Number of batches: ",len(train_loader))
for i, (x, y) in enumerate(train_loader):
    print(f"Batch {i}: samples {x.shape[0]}")

Number of batches:  4
Batch 0: samples 32
Batch 1: samples 32
Batch 2: samples 32
Batch 3: samples 31


DataLoader is designed to iterate over batches of data rather than individual samples, so when we try to access for example the first mini-batch, we get an error

However, note how before we could iterate over it

<span style="color:green">Yes, the cell below gives an error</span>



In [9]:
#train_loader[0]

# 2. Define the Neural Network

In [10]:
class RCNN(torch.nn.Module):
    
    def __init__(self):
        super(RCNN, self).__init__()

        # Define the layers

        # 1. Input layers
        #self.input_layer=torch.nn.Linear(in_features=166*166*30, out_features=166*166*30)      # not sure if this is needed
        #self.norm=torch.nn.BatchNorm2d(166*166*30)        #not sure this is sufficent for batch renormalization

        # 2. Convolutional layers

        # Three conv layers
        # according to gpt, in_channels=1 because in the sequence, it is still gray scale, not rgb
        self.conv1 = self.create_conv_layer(in_channels=30, out_channels=64)
        self.conv2 = self.create_conv_layer(in_channels=64, out_channels=64)
        self.conv3 = self.create_conv_layer(in_channels=64, out_channels=64)

        # First maxpool
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

        # Three conv layers
        self.conv4 = self.create_conv_layer(in_channels=64, out_channels=64)
        self.conv5 = self.create_conv_layer(in_channels=64, out_channels=64)
        self.conv6 = self.create_conv_layer(in_channels=64, out_channels=64)

        # Second maxpool
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

        # Flatten
        self.flatten=torch.nn.Flatten()

        # Each time we apply pooling, height and width are divided by 2
        self.flat_size = 64 * int(np.floor(166/2/2)) * int(np.floor(166/2/2))

        # Fully connected with dropout
        self.lin=torch.nn.Linear(self.flat_size, 1024)
        self.drop=torch.nn.Dropout(p=0.5)

        #LSTM
        # we set batch_first=True to have batch_size as first dimension: (30,166,166)
        self.lstm = torch.nn.LSTM(input_size=1024, hidden_size=512, num_layers=1, batch_first=True)

        #output: for binary classification softmax is not needed
        #self.output_layer=torch.nn.Softmax(dim=1)
        self.output = torch.nn.Linear(512, 1)

    # To make the code more clear, let's use this function to create conv layers
    def create_conv_layer(self, in_channels, out_channels):
        conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        relu = torch.nn.ReLU()
        norm = torch.nn.BatchNorm2d(out_channels)
        return torch.nn.Sequential(conv, relu, norm)



    def forward(self, x):
        
        # Define the forward pass

        # 1. Input layers
        #x=self.input_layer(x)
        #x=self.norm(x)

        # 2. Convolutional layers
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)

        # First maxpool
        x=self.pool1(x)

        # Three conv layers
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)

        # Second maxpool
        x=self.pool2(x)

        # Flatten
        x=self.flatten(x)

        # Fully connected with dropout
        x=self.lin(x)
        x=self.drop(x)

        #LSTM
        #x = x.view(x.size(0), -1, self.flat_size)
        #x = x.view(x.size(0), 30, -1)
        lstm_out, _ = self.lstm(x)
        #lstm_out = lstm_out[:, -1, :]

        #output
        out=self.output(lstm_out)
        
        return out.view(-1)

#### Define the model

<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> The code will crash here &darr; &darr; &darr; &darr; 
</div>


In [11]:
model = RCNN()

<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> The code will crash here &uarr; &uarr; &uarr; &uarr; 
</div>


# 3. Training Parameters

There is a lot that can be done here to get very cool hyperparameters

In [12]:
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 5

optimizer = torch.optim.AdamW(model.parameters())

criterion = torch.nn.functional.cross_entropy

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=(len(train_loader.dataset) * num_epochs) // train_loader.batch_size,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Adrien\anaconda3\envs\envADA\lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# 4. Train epoch script

In [13]:
def train_epoch(model, optimizer, scheduler, criterion, train_loader, epoch, device):
    model.train()
    loss_history = []
    accuracy_history = []
    lr_history = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()

        threshold = 0.5
        pred = (output >= threshold).float()
        correct = pred.eq(target.view_as(pred)).sum().item()
        loss_float = loss.item()
        accuracy_float = correct / len(data)

        loss_history.append(loss_float)
        accuracy_history.append(accuracy_float)
        lr_history.append(scheduler.get_last_lr()[0])
        
        # this is the if statement of the lab, but it doesn't work for me. it makes no sense to me tbh
        #if batch_idx % (len(train_loader.dataset) // len(data) // 10) == 0:
        # this doesn't work bad, but it prints twice per epoch
        if batch_idx == (len(train_loader.dataset) // len(data) - 1):
            print(
                f"Train Epoch: {epoch}-{batch_idx:03d} "
                f"batch_loss={loss_float:0.2e} "
                f"batch_acc={accuracy_float:0.3f} "
                f"lr={scheduler.get_last_lr()[0]:0.3e} "
            )

    return loss_history, accuracy_history, lr_history

# 4. Train the Network 😀😀😀😀😀


In [14]:
lr_history = []
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc, lrs = train_epoch(model, optimizer, scheduler, criterion, train_loader, epoch, device)
    train_loss_history.extend(train_loss)
    train_acc_history.extend(train_acc)
    lr_history.extend(lrs)

    # val_loss, val_acc = validate(model, device, val_loader, criterion)
    # val_loss_history.append(val_loss)
    # val_acc_history.append(val_acc)

Train Epoch: 1-002 batch_loss=1.05e+02 batch_acc=0.875 lr=9.397e-04 
Train Epoch: 1-003 batch_loss=9.67e+01 batch_acc=0.871 lr=8.946e-04 
Train Epoch: 2-002 batch_loss=1.05e+02 batch_acc=0.781 lr=7.008e-04 
Train Epoch: 2-003 batch_loss=9.63e+01 batch_acc=0.871 lr=6.227e-04 
Train Epoch: 3-002 batch_loss=1.08e+02 batch_acc=0.906 lr=3.773e-04 
Train Epoch: 3-003 batch_loss=9.35e+01 batch_acc=0.710 lr=2.992e-04 
Train Epoch: 4-002 batch_loss=1.12e+02 batch_acc=1.000 lr=1.054e-04 
Train Epoch: 4-003 batch_loss=9.25e+01 batch_acc=0.903 lr=6.026e-05 
Train Epoch: 5-002 batch_loss=1.00e+02 batch_acc=0.969 lr=0.000e+00 
Train Epoch: 5-003 batch_loss=1.03e+02 batch_acc=0.968 lr=6.819e-06 


# 5. Evaluate the Model

Validation set